In [39]:
import pandas as pd
import os
import numpy as np
import pickle as pkl
import copy
import matplotlib.pyplot as plt
import re
import scipy.stats as st
import copy
import itertools
# from statsmodels.stats.multitest import multipletests
import sys
sys.path.append(os.path.abspath(".."))
from torch import nn
import torch
from statsmodels.stats.multitest import multipletests
import json
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import re
from matplotlib.colors import LogNorm, Normalize
import ete3
from mditre_metabolites.plot_ete_tree import *
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Draw
from mditre_metabolites.plot_ete_tree import *
from mditre_metabolites.plot_results import *
%matplotlib inline
matplotlib.use('pdf')

In [53]:

dataset = 'franzosa'
path = f'/Users/jendawk/logs/aug9-FINAL-maybe/franzosa_cts/seqs_mets__F1=0d6726_F1_weighted=0d8586_AUC=0d9058/seed_6/'
# dataset = 'ibmdb'
# path = f'/Users/jendawk/logs/aug9-FINAL-maybe/ibmdb_cts/seqs_mets__F1=0d4168_F1_weighted=0d7206_AUC=0d6871/seed_4/'
dataset_tr = pd.read_pickle(path + '/train_dataset_used.pkl')
dataset_ts = pd.read_pickle(path + '/test_dataset_used.pkl')
rules = pd.read_csv(path + '/EVAL/rules.csv')
rule_activators = pd.read_pickle(path + '/EVAL/plotting_data/rule_activators.pkl')
detector_activators = pd.read_pickle(path + '/EVAL/plotting_data/detector_params.pkl')
k0 = list(detector_activators.keys())[0]
k1 = list(detector_activators[k0].keys())[0]
Y = detector_activators[k0][k1]['y']
ixs = Y.index.values

if Y.shape[0]>dataset_tr['metabs']['X'].shape[0]:
    X_metabs = pd.concat([dataset_tr['metabs']['X'], dataset_ts['metabs']['X']]).loc[ixs]
    X_otus = pd.concat([dataset_tr['otus']['X'], dataset_ts['otus']['X']]).loc[ixs]
else:
    X_metabs = dataset_tr['metabs']['X'].loc[ixs]
    X_otus = dataset_tr['otus']['X'].loc[ixs]
    


labels_map = {'franzosa': {1: 'Ctrl', 0: 'IBD'}, 'ibmdb': {1: 'Ctrl', 0: 'IBD'},
                                      'erawijantari': {0: 'Ctrl', 1: 'Gastrectomy'}, 'wang': {1: 'Ctrl', 0: 'ESRD'},
                                      'he_': {1: 'Breast Fed', 0: 'Formula Fed'},
                                      'cdi_': {1: 'CDI Recurrer', 0: 'Non-recurrer'},'semisyn':{0:'Ctrl',1:'Case'},
                                      }
dataset_paths = {'ibmdb':'/Users/jendawk/Dropbox (MIT)/microbes-metabolites/datasets/IBMDB/processed/ibmdb_pubchem/mets.pkl',
                 'franzosa':'/Users/jendawk/Dropbox (MIT)/microbes-metabolites/datasets/FRANZOSA/processed/franzosa_pubchem/mets.pkl'}

In [54]:

rule_locs=[]
det_params={}
limits={}
min_mets=np.inf
max_mets = -np.inf
min_otus = 1
max_otus = 0
for ix in rules.index.values:
    rule = rules.loc[ix].iloc[0]
    rule_int = int(rule.split(' ')[-1])
    if len(rule_locs)==0:
        rule_locs.append(rule_int)
    elif rule_int!=rule_locs[-1]:
        rule_locs.append(rule_int)
    if rule_int not in det_params.keys():    
        det_params[rule_int]={}
    det = rules.loc[ix].iloc[1]
    det_int = int(det.split(' ')[-1])
    features = rules.loc[ix]['features']
    feats_ls=features.split('[')[-1].split(']')[0].split("'")[1::2]
    if 'ASV' in feats_ls[0]:
        feats_key = [f.split(' (')[0] for f in feats_ls]
        data_1 = X_otus[feats_key].loc[Y==1]
        data_0 = X_otus[feats_key].loc[Y==0]
        dtype='otus'
        if X_otus[feats_key].values.flatten().min()<min_otus:
            min_otus = X_otus[feats_key].values.flatten().min()
        if X_otus[feats_key].values.flatten().max()>max_otus:
            max_otus = X_otus[feats_key].values.flatten().max()
    else:
        feats_key = feats_ls
        data_1 = X_metabs[feats_key].loc[Y==1]
        data_0 = X_metabs[feats_key].loc[Y==0]
        dtype='metabs'
        if X_metabs[feats_key].values.flatten().min()<min_mets:
            min_mets = X_metabs[feats_key].values.flatten().min()
        if X_metabs[feats_key].values.flatten().max()>max_mets:
            max_mets = X_metabs[feats_key].values.flatten().max()
            
        
    thresh = rules.loc[ix]['Detector Threshold']
    log_odds = rules.loc[ix]['Rule Log Odds']
    det_params[rule_int][det_int]={'feature_names':feats_ls,'threshold':thresh,'log_odds':log_odds, 
                                   'feature_key':feats_key, 'data_0':data_0.T, 'data_1':data_1.T,
                                  'activators':detector_activators[rule_int][det_int]['activators'],'dtype':dtype}

In [42]:
def axis_off(axes):
    for ax in axes:
        ax.axis('off')

def ticks_off(axes, left=False, labelleft=False, bottom=False, labelbottom=False):
    if isinstance(axes, np.ndarray):
        axes = axes.flatten()
    elif not isinstance(axes, list):
        axes = [axes]
    for ax in axes:
        ax.yaxis.set_tick_params(left=left, labelleft=labelleft)
        ax.xaxis.set_tick_params(bottom=bottom, labelbottom=labelbottom)

In [55]:
# cmap_taxa = plt.get_cmap('YlGnBu')
cmap_taxa = sns.color_palette('YlGnBu', as_cmap=True)
cmap_mets = sns.diverging_palette(145, 300, s=60, as_cmap=True)
# cmap_mets = sns.diverging_palette(220, 20, as_cmap=True)
cmap_mets.set_bad('w')
cmap_taxa.set_bad('w')
num_removed=0
last_row_0, last_row_1 = None, None
adj = True
for r in det_params.keys():
    r_act = rule_activators[r]
    for d in det_params[r].keys():
        n_rows = 1
        labels = copy.deepcopy(det_params[r][d]['feature_names'])
        data_0 = copy.deepcopy(det_params[r][d]['data_0'])
        # if data_0.shape[1]>80:
        #     num_removed = data_0.shape[1]-80
        #     data_0 = data_0.iloc[:, :80]
        data_1 = copy.deepcopy(det_params[r][d]['data_1'])
        #HILPn__
        mname = ['_____________' + m.split('__')[-1] for m in data_0.index.values]
        mnamedict = {m:'_____________' + m.split('__')[-1] for m in data_0.index.values}
        
        if len(mname)>len(set(mname)):
            data_0 = data_0.rename(index=mnamedict)
            data_0 = data_0.groupby(data_0.index).mean()
            data_1 = data_1.groupby(data_1.index).mean()
        print('dtype', det_params[r][d]['dtype'])
        if det_params[r][d]['dtype']=='metabs':
            # vmin,vmax = min_mets, max_mets
            cmap=cmap_mets
            if data_0.shape[0]>1:
                n_rows = 2
                data_0.loc[' ']=[np.nan]*data_0.shape[1]
                data_1.loc[' ']=[np.nan]*data_1.shape[1]
                data_0.loc['  ']=[np.nan]*data_0.shape[1]
                data_1.loc['  ']=[np.nan]*data_1.shape[1]
                data_0.loc['Average']=data_0.mean(0)
                data_1.loc['Average']=data_1.mean(0)
                labels.append('Average')
        else:
            
            # vmin,vmax = min_otus, max_otus
            cmap = cmap_taxa
            if data_0.shape[0]>1:
                n_rows = 2
                data_0.loc[' ']=[np.nan]*data_0.shape[1]
                data_1.loc[' ']=[np.nan]*data_1.shape[1]
                # data_0.loc['  ']=[np.nan]*data_0.shape[1]
                # data_1.loc['  ']=[np.nan]*data_1.shape[1]
                data_0.loc['Sum']=data_0.sum(0)
                data_1.loc['Sum']=data_1.sum(0)
                labels.append('Sum')
        
        # print(data_0)
        if det_params[r][d]['dtype']!='metabs':
            # data_0 = data_0 + 1e-10
            # data_1 = data_1 + 1e-10
            # norm=LogNorm()
            norm=None
            if adj: 
                tmp = 2.5
                adj=False
            else:
                tmp = 1.75
                
            # fig, ax = plt.subplots(figsize = (6,len(det_params[r][d]['feature_key'])/tmp))
            # ax, feat_order = plot_phylo_tree(dataset_tr['otus'], det_params[r][d]['feature_key'], ax=ax,
            #                        out_path=f'{path}/rule_{r}_detector_{d}_tree.nw')
            # fig.savefig(f'/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/F4_RealExample1/tree_rule_{r}_detector_{d}.pdf',
            #             transparent=True, bbox_inches='tight')
        else:
            metabolites = [l.split('__')[-1] for l in labels]
            if len(metabolites)>1:
                metabolites = metabolites[:-1]
            # print('structures',metabolites)
            norm=None
            # print([l.split('__')[-1] for l in labels])
            dataset_path = dataset_paths[dataset]
            dataset_path = dataset_path.replace('infomax', 'pubchem').replace('morgan', 'pubchem').replace('mqn',
                                                                                                   'pubchem').replace('map4', 'pubchem')
            ikeys = pd.read_csv('/'.join(dataset_path.split('/')[:-1])
                                + '/metabolite_fingerprints.csv', index_col=0)
            ikeys.index = [m.split('__')[-1].split(": ")[-1] for m in ikeys.index.values]
            # print(ikeys.index.values)
            if all([m not in ikeys.index.values for m in metabolites]):
                mets = ikeys.index.values
                m_rename = {m: '__'.join([m.split('_')[0], m.split('__')[-1].split(': ')[-1]]) for m in mets}
                ikeys = ikeys.rename(index=m_rename)
                # print(ikeys.index.values)
            inchis = ikeys['inchikey'].loc[metabolites].drop_duplicates().to_dict()
            if 'hmdb' in ikeys.columns.values:
                hmdbs = ikeys['hmdb'].loc[metabolites].drop_duplicates().to_dict()
            else:
                print('No hmdbs - hmdbs coming soon!')
                hmdbs = {}
            ms, names, imgs = [], [], []
            for k, ikey in inchis.items():
                out = pcp.get_compounds(ikey, namespace='inchikey')[0]
                # except:
                #     continue
                m = Chem.MolFromInchi(out.inchi)
                ms.append(m)
                names.append(k)
                imgs.append(Draw.MolToImage(m))
            # imgs = plot_structures([l.split('__')[-1] for l in labels][:-1], dataset_paths[dataset])
            if len(imgs) > 0:
                
                for i in range(len(imgs)):
                    # img = plot_structures(feats_to_plot, dpath_dict['metabs'])
                    fig, ax = plt.subplots(figsize = (4,4))
                    try:
                        img_arr = np.asarray(imgs[i])
                    except:
                        continue
                    ax.imshow(img_arr);
                    ax.spines['right'].set_visible(False)
                    ax.spines['bottom'].set_visible(False)
                    ax.spines['left'].set_visible(False)
                    ax.xaxis.set_tick_params(labelbottom=False, bottom=False)
                    ax.yaxis.set_tick_params(labelleft=False, left=False)
                    # ax.set_title(labels[i], fontsize=18)
                    fig.savefig(f'/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/Figure5-CaseStudies/Links/{labels[i]}.jpeg',
                                bbox_inches='tight', dpi=1000)
                    plt.close(fig)

    
        N = data_0.shape[0]
        
        S = data_0.shape[1] + data_1.shape[1]
        print(N)
        f = 10

        h = N*f/72
        w = ((S-2)*h)/N
        fig, ax = plt.subplots(1, 3,figsize=(w, h),
                       gridspec_kw={
                           'wspace': 0.000001,
                           'width_ratios': [sum(Y == 0), sum(Y == 1),1],},)
        # if n_rows == 1:
        #     ax = np.expand_dims(ax, 0)
        vmin = min([data_0.min().min(), data_1.min().min()])
        # if vmin ==0:
        #     vmin = np.spacing(0.0)
        vmax = max([data_0.max().max(), data_1.max().max()])
        if det_params[r][d]['dtype']=='metabs':
            vmax =4
            vmin =-2
        else:
            if vmax>.05:
                vmax=0.05
        ticks_off(ax[0], labelleft=True)
        ticks_off(ax[1])
        # ticks_off(ax)
        ticklabels= [det_params[r][d]['threshold']]
            # data_0.loc['tryptophan'] = data_0.loc['tryptophan'].mean(0)
            

        sns.heatmap(data_0.values, norm=norm, ax=ax[0], cbar=False, square=True, cmap = cmap, center = det_params[r][d]['threshold'], xticklabels=False, vmin=vmin, vmax=vmax,
                   yticklabels=data_0.index.values)
        sns.heatmap(data_1.values, norm=norm, ax=ax[1], cbar_ax = ax[2], cbar_kws={'ticks':ticklabels}, square=True, cmap = cmap, center = det_params[r][d]['threshold'], 
                    xticklabels=False, vmin=vmin, vmax=vmax)
        bbox = ax[0].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        width, height = bbox.width, bbox.height
        fsize_y = (height/data_0.shape[0])*72 
        ax[0].tick_params(axis='y',labelsize=fsize_y)
        ax[2].tick_params(axis='y',labelsize=fsize_y)


        # if n_rows>1:
        #     sns.heatmap(last_row_0, ax=ax[-1,0], cbar=False, square=True, cmap = cmap, center = det_params[r][d]['threshold'], xticklabels=False, vmin=vmin, vmax=vmax)
        #     sns.heatmap(last_row_1, ax=ax[-1,1], square=True, cmap = cmap, center = det_params[r][d]['threshold'], xticklabels=False, vmin=vmin, vmax=vmax,
        #                cbar_ax = ax[:,2], cbar_kws={'ticks':ticklabels})
            
        # fig.savefig(f'rule_{r}_detector_{d}_no_labels.pdf',transparent=True, bbox_inches='tight')
        # fig.show()
        plt.close(fig)
        # ax[0].set_xlabel(labels_map[dataset][0])
        # ax[1].set_xlabel(labels_map[dataset][1])
        # fig.savefig(f'/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/F4_RealExample1/rule_{r}_detector_{d}.pdf',transparent=True, bbox_inches='tight')

dtype metabs
1
dtype metabs
1
dtype otus
37


In [19]:
cmap_act = sns.light_palette('tomato', as_cmap=True)

num_detectors = len(rules.index.values)
num_rules = len(rule_locs)
# fig_act, ax_act = plt.subplots(num_detectors + num_rules*2, 2,
#                                gridspec_kw={'hspace': 0.05, 'wspace': 0.01,
#                                             'width_ratios': [sum(Y == 0), sum(Y == 1)],
#                                             'height_ratios': [1] * (num_detectors + num_rules * 2 )},
#                                figsize=(8.5,(num_detectors + num_rules * 2 ) * 0.3), constrained_layout=True)
# ticks_off(ax_act)
ix_act = 0
for ir,r in enumerate(det_params.keys()):
    r_act=np.prod(np.vstack([(1 - (1 - det_params[r][i]['activators'].squeeze())) for i in det_params[r].keys()]), 0)
    print(f'rule {r}')
    num_removed=0
    for id,d in enumerate(det_params[r].keys()):
        d_act = det_params[r][d]['activators']
        if len(d_act[Y.values==0])>80:
            num_removed = len(d_act[Y.values==0])-80
            d_act_0 = d_act[Y.values==0][:80]
            r_act_0 = r_act[Y.values==0][:80]
        else:
            d_act_0 = d_act[Y.values==0]
            r_act_0 = r_act[Y.values==0]
        fig_act, ax_act = plt.subplots(1, 3, gridspec_kw={'hspace': 0.05, 'wspace': 0.01,
                                            'width_ratios': [sum(Y == 0)-num_removed, sum(Y == 1), 1.5]},
                                       figsize=(10,0.5*np.log(2)), constrained_layout=True)
        ticks_off(ax_act)
                                       
        sns.heatmap(np.expand_dims(d_act_0,0), ax=ax_act[0], cbar=False, square=True, cmap = cmap_act, center = 0.5, vmin=0, vmax=1)
        sns.heatmap(np.expand_dims(d_act[Y.values==1],0), ax=ax_act[1], cbar_ax = ax[2], cbar_kws={'ticks':ticklabels}, 
                    square=True, cmap = cmap_act, center = 0.5, vmin=0, vmax=1)
        fig_act.savefig(f'/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/F4_RealExample1/rule_{r}_detector_{d}_activation.pdf',transparent=True, bbox_inches='tight')
        # ax_act[0].set_yticks([0.5],[f'Detector {id}'], rotation=0)
        ix_act += 1
        print(f'detector {d}')

    fig_act, ax_act = plt.subplots(1, 3, gridspec_kw={'hspace': 0.05, 'wspace': 0.01,
                                    'width_ratios': [sum(Y == 0)-num_removed, sum(Y == 1), 1.5]},
                               figsize=(10,0.5*np.log(2)), constrained_layout=True)
    ticks_off(ax_act)
    sns.heatmap(np.expand_dims(r_act_0,0), ax=ax_act[0], cbar=False, square=True, cmap = cmap_act, center = 0.5, vmin=0, vmax=1)
    sns.heatmap(np.expand_dims(r_act[Y.values==1],0), ax=ax_act[1], cbar_ax = ax[2], cbar_kws={'ticks':ticklabels},
                square=True, cmap = cmap_act, center = 0.5, vmin=0, vmax=1)
    # ax_act[ix_act,0].set_yticks([0.5], [f'Rule {ir}'], rotation=0)
    # axis_off(ax_act[ix_act+1,:])
    
    ix_act += 2
    # fig_act.savefig(f'/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/F4_RealExample1/rule_{r}_activation.pdf',transparent=True, bbox_inches='tight')

rule 0


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jendawk/Dropbox (MIT)/MDITRE-Metabolites Paper/FINAL_FIGURES/F4_RealExample1/rule_0_detector_10_activation.pdf'